In [1]:
import sys # required for relative imports in jupyter lab
sys.path.insert(0, '../') 

from scipy.sparse import coo_matrix
import numpy as np

from dataset import QM9

from torch_geometric.datasets import QM9 as TQM9
from torch_geometric.data import Data



RDKit WARNING: [22:03:05] Enabling RDKit 2019.09.3 jupyter extensions


In [2]:
ds_params = {'train_params': {'n': 1000,
                              'db': 'rdkit',
                              'n_conformers': 1,
                              'features': ['n_atoms','A','B','C','mu',
                                           'alpha','homo','lumo','gap','r2','zpve','Cv',
                                           'mulliken','distance','coulomb','aromatic',
                                           'degree'],
                              'embeds': ['atom_types','chirality','hybridization'],
                              'targets': ['U0'],
                              'as_dict': True,
                              'pad': None,
                              'do_not_pad': ['U0','n_atoms','A','B','C','mu',
                                             'alpha','homo','lumo','gap','r2','zpve','Cv'],
                              #'filter_on': ('n_atoms','>','18'),
                              'use_pickle': False,
                              'flatten': True,
                              'embed_lookup': {'0': 0, 'H': 1, 'C': 2, 'N': 3, 'O': 4, 'F': 5,
                                               'CHI_UNSPECIFIED': 6, 'CHI_TETRAHEDRAL_CW': 7,
                                               'CHI_TETRAHEDRAL_CCW':8, 'CHI_OTHER': 9, 
                                               'UNSPECIFIED': 10, 'S': 11, 'SP': 12, 
                                               'SP2': 13, 'SP3': 14, 'SP3D': 15, 'SP3D2': 16,
                                               'OTHER': 17, 'misc': 18,}}}

qm9 = QM9(**ds_params['train_params'])

creating QM9 dataset...
molecules scanned:  1
molecules removed for no rdkit conformer:  0
molecules created:  1
molecules scanned:  1001
molecules removed for no rdkit conformer:  15
molecules created:  986
total molecules scanned:  1015
total molecules removed for no rdkit conformer:  15
total uncharacterized molecules removed:  26
total molecules created:  974
CDataset created...


In [3]:
mol = qm9.ds[5]
mol

dsgdb9nsd_000005

In [4]:
qm9[5]

{'X': array([ 3.00000000e+00,  0.00000000e+00,  4.45938835e+01,  4.45938835e+01,
         2.89369988e+00,  1.29899998e+01, -3.60399991e-01,  1.90999992e-02,
         3.79599988e-01,  4.87476006e+01,  1.66010000e-02,  6.27799988e+00,
        -4.96560000e-02, -1.88473001e-01,  2.38128006e-01,  0.00000000e+00,
         1.16098726e+00,  1.06317782e+00,  1.16098726e+00,  0.00000000e+00,
         2.22408032e+00,  1.06317782e+00,  2.22408032e+00,  0.00000000e+00,
         5.33587189e+01,  2.43947468e+01,  3.07678947e+01,  1.68529167e+01,
         3.68581123e+01,  2.63724823e+01,  2.80881977e+00,  3.48496389e+00,
         5.00000000e-01,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         2.00000000e+00,  1.00000000e+00,  1.00000000e+00]),
 'embed_idx': [array([2, 3, 1]), array([6, 6, 6]), array([12, 12, 10])],
 'y': array([-93.41189], dtype=float32)}

In [5]:
mol.edge_indices

array([[0, 1],
       [1, 0],
       [0, 2],
       [2, 0]])

In [ ]:
mol.edge_attr

In [ ]:
mol.rdmol.GetNumConformers()

In [ ]:
for f in dir(mol):
    print(f)
    print(getattr(mol, f))

In [ ]:
mol.rdmol